<a href="https://colab.research.google.com/github/iocariz/stock_market_analytics/blob/main/Module2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

In [2]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

**Question 1: IPO Filings Web Scraping and Data Processing**
What's the total sum ($m) of 2023 filings that happenned of Fridays?

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/

Convert the 'Filing Date' to datetime()
'Shares Offered' to float64 (if '-' is encountered, populate with NaNs)
Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given)You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string.
Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)
Find the total sum in $m (millions of USD, closest INTEGER number) for all fillings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 24 of it is not null.
(additional: you can read about S-1 IPO filing to understand the context)

In [3]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [4]:
ipo_fillings = ipo_dfs[0]

ipo_fillings.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     326 non-null    object
 1   Symbol          326 non-null    object
 2   Company Name    326 non-null    object
 3   Price Range     326 non-null    object
 4   Shares Offered  326 non-null    object
dtypes: object(5)
memory usage: 12.9+ KB


In [5]:
ipo_fillings.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-


In [6]:
import numpy as np
ipo_fillings['Shares Offered'] = ipo_fillings['Shares Offered'].apply(lambda x: float(x) if x != '-' else np.nan)

In [9]:
# Convert 'Filing Date' to datetime
ipo_fillings['Filing Date'] = pd.to_datetime(ipo_fillings['Filing Date'], format='mixed')

In [11]:
#not always the columns are filled
missing_prices_df = ipo_fillings[ipo_fillings['Price Range'].astype(str).str.find('-') >= 0]
missing_prices_df

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN
8,2024-04-15,GAUZ,Gauzy Ltd.,-,NaN
9,2024-04-12,BOW,Bowhead Specialty Holdings Inc.,-,NaN
10,2024-04-05,SPHL,Springview Holdings Ltd,$4.00 - $5.00,2000000.0
...,...,...,...,...,...
319,2020-06-17,ALEH,ALE Group Holding Limited,$4.00 - $6.00,4200000.0
320,2020-02-18,CAST,"FreeCast, Inc.",-,NaN
321,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0
323,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0


In [12]:
def str_to_digits(s):
    if not s or s == '-':  # If empty string or just a dash
        return np.nan
    s = s.replace('$', '').replace(',', '')  # Remove $ and commas
    if '-' in s:  # If it's a range
        try:
            start, end = map(float, s.split('-'))
            return (start + end) / 2  # Return average of the range
        except ValueError:
            return np.nan  # Return nan if conversion fails
    else:
        try:
            return float(s)  # Convert to float
        except ValueError:
            return np.nan  # Return nan if conversion fails

# Apply the function to the 'Price Range' column
ipo_fillings['Avg_price'] = ipo_fillings['Price Range'].apply(lambda x: str_to_digits(x))

print(ipo_fillings)

    Filing Date Symbol                           Company Name     Price Range  \
0    2024-05-03    TBN         Tamboran Resources Corporation               -   
1    2024-04-29   HWEC                   HW Electro Co., Ltd.           $3.00   
2    2024-04-29   DTSQ  DT Cloud Star Acquisition Corporation          $10.00   
3    2024-04-26   EURK                Eureka Acquisition Corp          $10.00   
4    2024-04-26    HDL    Super Hi International Holding Ltd.               -   
..          ...    ...                                    ...             ...   
321  2020-01-21   GOXS                            Goxus, Inc.  $8.00 - $10.00   
322  2020-01-21   UTXO                 UTXO Acquisition, Inc.          $10.00   
323  2019-12-09   LOHA                           Loha Co. Ltd  $8.00 - $10.00   
324  2019-10-04   ZGHB  China Eco-Materials Group Co. Limited           $4.00   
325  2018-12-27   FBOX              Fit Boxx Holdings Limited   $5.00 - $6.50   

     Shares Offered  Avg_pr

In [13]:
# Define a column "Shares_offered_value",
# which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)
def calculate_shares_offered_value(row):
    shares_offered = row['Shares Offered']
    avg_price = row['Avg_price']
    if pd.notnull(shares_offered) and pd.notnull(avg_price):
        return shares_offered * avg_price
    else:
        return np.nan

# Apply the function to create the Shares_offered_value column
ipo_fillings['Shares_offered_value'] = ipo_fillings.apply(calculate_shares_offered_value, axis=1)

print(ipo_fillings)

    Filing Date Symbol                           Company Name     Price Range  \
0    2024-05-03    TBN         Tamboran Resources Corporation               -   
1    2024-04-29   HWEC                   HW Electro Co., Ltd.           $3.00   
2    2024-04-29   DTSQ  DT Cloud Star Acquisition Corporation          $10.00   
3    2024-04-26   EURK                Eureka Acquisition Corp          $10.00   
4    2024-04-26    HDL    Super Hi International Holding Ltd.               -   
..          ...    ...                                    ...             ...   
321  2020-01-21   GOXS                            Goxus, Inc.  $8.00 - $10.00   
322  2020-01-21   UTXO                 UTXO Acquisition, Inc.          $10.00   
323  2019-12-09   LOHA                           Loha Co. Ltd  $8.00 - $10.00   
324  2019-10-04   ZGHB  China Eco-Materials Group Co. Limited           $4.00   
325  2018-12-27   FBOX              Fit Boxx Holdings Limited   $5.00 - $6.50   

     Shares Offered  Avg_pr

In [14]:
friday_fillings = ipo_fillings[(ipo_fillings['Filing Date'].dt.dayofweek == 4) & (ipo_fillings['Filing Date'].dt.year == 2023)]
total_sum_in_millions = int(friday_fillings['Shares_offered_value'].sum() / 1000000)
print(f"Total sum of 2023 filings that happened on Fridays: ${total_sum_in_millions}m")

Total sum of 2023 filings that happened on Fridays: $285m


**Question 2: IPOs "Fixed days hold" strategy
Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**
Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Some of the tickers like 'DYCQ' and 'LEGT' were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:

You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset. You can use the DataFrame.describe() function to get mean, min, max, 25-50-75% quantiles. Addtional:

You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%. What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

In [15]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [16]:
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [17]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)


In [18]:
ipos_2024 = ipo_dfs[0]
ipos_2024.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      67 non-null     object
 1   Symbol        67 non-null     object
 2   Company Name  67 non-null     object
 3   IPO Price     67 non-null     object
 4   Current       67 non-null     object
 5   Return        67 non-null     object
dtypes: object(6)
memory usage: 3.3+ KB


In [19]:
# "stacking dataframes"
# pd.concat() is used to concatenate the DataFrames vertically.
# The ignore_index=True parameter ensures that the resulting DataFrame has a new index,
# ignoring the original indices of the input DataFrames.
# The stacked_df now contains the concatenated DataFrame.
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

In [20]:
stacked_ipos_df.head(1)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 9, 2024",SVCO,"Silvaco Group, Inc.",$19.00,$19.25,7.68%


In [21]:
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      221 non-null    object
 1   Symbol        221 non-null    object
 2   Company Name  221 non-null    object
 3   IPO Price     221 non-null    object
 4   Current       221 non-null    object
 5   Return        221 non-null    object
dtypes: object(6)
memory usage: 10.5+ KB


In [22]:
missing_prices_df = stacked_ipos_df[stacked_ipos_df['IPO Price'].astype(str).str.find('-') >= 0]
missing_prices_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return


In [23]:
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'].str.replace('$', ''), errors='coerce')
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'])

In [24]:
# Convert "Current" column
stacked_ipos_df['Current'] = pd.to_numeric(stacked_ipos_df['Current'].str.replace('$', ''), errors='coerce')

# Convert 'Return' to numeric format (percentage)
stacked_ipos_df['Return'] = pd.to_numeric(stacked_ipos_df['Return'].str.replace('%', ''), errors='coerce') / 100

In [25]:
stacked_ipos_df.isnull().sum()

IPO Date        0
Symbol          0
Company Name    0
IPO Price       0
Current         0
Return          2
dtype: int64

In [26]:
stacked_ipos_df[stacked_ipos_df.Return.isnull()]

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
7,"Apr 25, 2024",MRX,Marex Group plc,19.00,19.19,NaN
199,"Feb 15, 2023",GXAI,Gaxos.ai Inc.,4.15,4.10,NaN


In [27]:
stacked_ipos_df['IPO Price'].mean()

11.082036199095024

In [28]:
print(stacked_ipos_df)

         IPO Date Symbol                             Company Name  IPO Price  \
0     May 9, 2024   SVCO                      Silvaco Group, Inc.       19.0   
1     May 9, 2024    PAL          Proficient Auto Logistics, Inc.       15.0   
2     May 8, 2024    NNE                 Nano Nuclear Energy Inc.        4.0   
3     May 1, 2024    VIK                     Viking Holdings Ltd.       24.0   
4    Apr 26, 2024   ZONE                CleanCore Solutions, Inc.        4.0   
..            ...    ...                                      ...        ...   
216  Jan 25, 2023    QSG                     QuantaSing Group Ltd       12.5   
217  Jan 20, 2023   CVKD              Cadrenal Therapeutics, Inc.        5.0   
218  Jan 13, 2023   SKWD  Skyward Specialty Insurance Group, Inc.       15.0   
219  Jan 13, 2023   ISRL                 Israel Acquisitions Corp       10.0   
220  Jan 13, 2023   MGOL                          MGO Global Inc.        5.0   

     Current  Return  
0      19.25  0.

In [29]:
formats = ["%b %d, %Y", "%Y-%m-%d", "%m/%d/%Y", "%B %d, %Y"]
for fmt in formats:
    try:
        stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format=fmt)
        break  # Stop looping if conversion succeeds
    except ValueError:
        continue  # Continue to the next format if conversion fails

print(stacked_ipos_df)

      IPO Date Symbol                             Company Name  IPO Price  \
0   2024-05-09   SVCO                      Silvaco Group, Inc.       19.0   
1   2024-05-09    PAL          Proficient Auto Logistics, Inc.       15.0   
2   2024-05-08    NNE                 Nano Nuclear Energy Inc.        4.0   
3   2024-05-01    VIK                     Viking Holdings Ltd.       24.0   
4   2024-04-26   ZONE                CleanCore Solutions, Inc.        4.0   
..         ...    ...                                      ...        ...   
216 2023-01-25    QSG                     QuantaSing Group Ltd       12.5   
217 2023-01-20   CVKD              Cadrenal Therapeutics, Inc.        5.0   
218 2023-01-13   SKWD  Skyward Specialty Insurance Group, Inc.       15.0   
219 2023-01-13   ISRL                 Israel Acquisitions Corp       10.0   
220 2023-01-13   MGOL                          MGO Global Inc.        5.0   

     Current  Return  
0      19.25  0.0768  
1      14.79 -0.0090  
2     

In [30]:
# Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01")
# - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

ipo_filter = stacked_ipos_df[stacked_ipos_df['IPO Date'] <= '2024-03-01']
ipo_filter = ipo_filter[ipo_filter.Symbol != 'RYZB']
ipo_filter

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
36,2024-02-27,SMXT,"SolarMax Technology, Inc.",4.00,10.88,1.7125
37,2024-02-22,VHAI,Vocodia Holdings Corp,4.25,0.14,-0.9673
38,2024-02-21,DYCQ,DT Cloud Acquisition Corporation,10.00,10.16,0.0160
39,2024-02-16,CHRO,Chromocell Therapeutics Corp,6.00,1.75,-0.7083
40,2024-02-14,UMAC,"Unusual Machines, Inc.",4.00,1.10,-0.7375
...,...,...,...,...,...,...
216,2023-01-25,QSG,QuantaSing Group Ltd,12.50,3.18,-0.7480
217,2023-01-20,CVKD,"Cadrenal Therapeutics, Inc.",5.00,0.48,-0.9040
218,2023-01-13,SKWD,"Skyward Specialty Insurance Group, Inc.",15.00,37.26,1.4603
219,2023-01-13,ISRL,Israel Acquisitions Corp,10.00,10.93,0.0930


In [31]:
symbols = ipo_filter['Symbol'].tolist()
symbols[symbols.index('PTHR')] = 'PTHRF'
symbols

['SMXT',
 'VHAI',
 'DYCQ',
 'CHRO',
 'UMAC',
 'TBBB',
 'MGX',
 'HLXB',
 'TELO',
 'KYTX',
 'PMNT',
 'AHR',
 'LEGT',
 'ANRO',
 'GUTS',
 'AS',
 'FBLG',
 'BTSG',
 'AVBP',
 'HAO',
 'CGON',
 'YIBO',
 'SUGP',
 'JL',
 'KSPI',
 'JVSA',
 'PSBD',
 'CCTG',
 'SYNX',
 'SDHC',
 'ROMA',
 'IROH',
 'LGCB',
 'ZKH',
 'BAYA',
 'INHD',
 'AFJK',
 'GSIW',
 'FEBO',
 'CLBR',
 'ELAB',
 'RR',
 'DDC',
 'SHIM',
 'GLAC',
 'SGN',
 'HG',
 'CRGX',
 'ANSC',
 'AITR',
 'GVH',
 'LXEO',
 'PAPL',
 'ATGL',
 'MNR',
 'WBUY',
 'NCL',
 'BIRK',
 'GMM',
 'PMEC',
 'LRHC',
 'GPAK',
 'SPKL',
 'QETA',
 'MSS',
 'ANL',
 'SYRA',
 'VSME',
 'LRE',
 'TURB',
 'MDBH',
 'KVYO',
 'CART',
 'DTCK',
 'NMRA',
 'ARM',
 'SPPL',
 'NWGL',
 'SWIN',
 'IVP',
 'NNAG',
 'SRM',
 'SPGC',
 'LQR',
 'NRXS',
 'FTEL',
 'MIRA',
 'PXDT',
 'HRYU',
 'CTNT',
 'SRFM',
 'PRZO',
 'HYAC',
 'KVAC',
 'JNVR',
 'ELWS',
 'WRNT',
 'TSBX',
 'ODD',
 'APGE',
 'NETD',
 'SGMT',
 'BOWN',
 'SXTP',
 'PWM',
 'VTMX',
 'INTS',
 'SVV',
 'KGS',
 'FIHL',
 'GENK',
 'BUJA',
 'BOF',
 'AZTR',
 'CA

In [32]:
# Get OHLCV daily prices for all stocks
import yfinance as yf

prices = {}
for i in symbols:
    try:
        prices[i] = yf.download(i, start='2023-01-01', interval = "1d")
    except:
        pass

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [33]:
# Create a list of DataFrames for each stock
data_frames = []
for symbol, data in prices.items():
    # Create a DataFrame for each stock
    df_stock = pd.DataFrame(data)
    df_stock['Symbol'] = symbol  # Add a 'Symbol' column
    data_frames.append(df_stock)

# Concatenate the DataFrames into a single DataFrame
df = pd.concat(data_frames)
df.reset_index(inplace=True)
# Display the DataFrame info
print(df)

            Date    Open    High     Low   Close  Adj Close   Volume Symbol
0     2024-02-27  3.5000  9.3500  3.5000  8.0000     8.0000  2840400   SMXT
1     2024-02-28  6.4200  7.2400  5.6000  5.8600     5.8600   243200   SMXT
2     2024-02-29  5.7400  5.8400  4.3000  4.8900     4.8900   130000   SMXT
3     2024-03-01  3.9400  5.1500  3.8000  4.8800     4.8800   485000   SMXT
4     2024-03-04  4.7300  5.1000  4.2580  4.6600     4.6600   114300   SMXT
...          ...     ...     ...     ...     ...        ...      ...    ...
34901 2024-05-03  0.2260  0.2440  0.2210  0.2320     0.2320    63300   MGOL
34902 2024-05-06  0.2340  0.2400  0.2210  0.2210     0.2210   116000   MGOL
34903 2024-05-07  0.2320  0.2400  0.2250  0.2250     0.2250    46300   MGOL
34904 2024-05-08  0.2250  0.2370  0.2210  0.2290     0.2290    12800   MGOL
34905 2024-05-09  0.2299  0.2299  0.1905  0.2058     0.2058   251743   MGOL

[34906 rows x 8 columns]


In [34]:
# Get the first day when each stock has data on Yahoo Finance.
min_dates = df.min(axis=0)

# Generate 30 additional columns: growth_future_1d ... growth_future_30d.
for i in range(1, 31):
    df['growth_future_' + str(i) + 'd'] = df.shift(-i)['Adj Close'] / df['Adj Close'] - 1

In [35]:
for df in data_frames:
    for i in range(1,31):
        df['growth_future_'+str(i)+'d'] = df['Adj Close'] / df['Adj Close'].shift(i)

In [36]:
# Find the first day when each stock has data on Yahoo Finance
min_dates = df.min(axis=0)
min_dates = min_dates[1:]  # Remove the 'Date' column
min_dates_df = pd.DataFrame(data={'Symbol': df.columns[1:], 'min_date': min_dates})
min_dates_df.set_index('Symbol', inplace=True)
min_dates_df

,min_date
Symbol,
High,0.2299
Low,0.1905
Close,0.2058
Adj Close,0.2058
Volume,8800
Symbol,MGOL
growth_future_1d,0.526549
growth_future_2d,0.522124
growth_future_3d,0.469027


In [37]:

# Find X, when the 75% quantile growth is the highest
growth_quantiles = df.describe().loc['75%', :]

In [38]:
# Find X, when the 75% quantile growth is the highest
growth_quantiles = df.describe().loc['75%', :]
growth_quantiles = growth_quantiles[['growth_future_1d', 'growth_future_2d', 'growth_future_3d', 'growth_future_4d', 'growth_future_5d', 'growth_future_6d', 'growth_future_7d', 'growth_future_8d', 'growth_future_9d', 'growth_future_10d', 'growth_future_11d', 'growth_future_12d', 'growth_future_13d', 'growth_future_14d', 'growth_future_15d', 'growth_future_16d', 'growth_future_17d', 'growth_future_18d', 'growth_future_19d', 'growth_future_20d', 'growth_future_21d', 'growth_future_22d', 'growth_future_23d', 'growth_future_24d', 'growth_future_25d', 'growth_future_26d', 'growth_future_27d', 'growth_future_28d', 'growth_future_29d', 'growth_future_30d']]
growth_quantiles = pd.to_numeric(growth_quantiles)
optimal_x = growth_quantiles.idxmax()
optimal_x_value = growth_quantiles.loc[optimal_x]
print(f"Optimal X: {optimal_x}")
print(f"75% quantile growth for optimal X: {optimal_x_value:.2%}")

Optimal X: growth_future_3d
75% quantile growth for optimal X: 105.08%


In [39]:
# Addtional: You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
mean_growth = df.describe().loc['mean', :]
mean_growth = mean_growth[['growth_future_1d', 'growth_future_2d', 'growth_future_3d', 'growth_future_4d', 'growth_future_5d', 'growth_future_6d', 'growth_future_7d', 'growth_future_8d', 'growth_future_9d', 'growth_future_10d', 'growth_future_11d', 'growth_future_12d', 'growth_future_13d', 'growth_future_14d', 'growth_future_15d', 'growth_future_16d', 'growth_future_17d', 'growth_future_18d', 'growth_future_19d', 'growth_future_20d', 'growth_future_21d', 'growth_future_22d', 'growth_future_23d', 'growth_future_24d', 'growth_future_25d', 'growth_future_26d', 'growth_future_27d', 'growth_future_28d', 'growth_future_29d', 'growth_future_30d']]
median_growth = df.describe().loc['50%', :]
median_growth = median_growth[['growth_future_1d', 'growth_future_2d', 'growth_future_3d', 'growth_future_4d', 'growth_future_5d', 'growth_future_6d', 'growth_future_7d', 'growth_future_8d', 'growth_future_9d', 'growth_future_10d', 'growth_future_11d', 'growth_future_12d', 'growth_future_13d', 'growth_future_14d', 'growth_future_15d', 'growth_future_16d', 'growth_future_17d', 'growth_future_18d', 'growth_future_19d', 'growth_future_20d', 'growth_future_21d', 'growth_future_22d', 'growth_future_23d', 'growth_future_24d', 'growth_future_25d', 'growth_future_26d', 'growth_future_27d', 'growth_future_28d', 'growth_future_29d', 'growth_future_30d']]
negative_returns_count = (mean_growth < 0).sum() + (median_growth < 0).sum()
print(f"Number of negative mean and median returns: {negative_returns_count}")
# What's your recommendation: Do you suggest pursuing this strategy for an optimal X?
# The optimal X is 1, which means buying a new stock (listed on IPO) on the first day and selling it on the next day

Number of negative mean and median returns: 0


**Question 3: Is Growth Concentrated in the Largest Stocks?
Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**

Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

In [40]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']


In [41]:
LARGEST_STOCKS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
LARGEST_STOCKS

['MSFT',
 'AAPL',
 'GOOG',
 'NVDA',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'NVO',
 'MC.PA',
 'ASML',
 'RMS.PA',
 'OR.PA',
 'SAP',
 'ACN',
 'TTE',
 'SIE.DE',
 'IDEXY',
 'CDI.PA',
 'RELIANCE.NS',
 'TCS.NS',
 'HDB',
 'BHARTIARTL.NS',
 'IBN',
 'SBIN.NS',
 'LICI.NS',
 'INFY',
 'ITC.NS',
 'HINDUNILVR.NS',
 'LT.NS']

In [43]:

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [44]:
import time

largest_stocks_df = pd.DataFrame({'A' : []})
# Define the start and end dates
start_date = '2013-12-25'
end_date = '2024-01-01'
for i,ticker in enumerate(LARGEST_STOCKS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,start=start_date, end=end_date,
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['ticker_category']= "LARGEST"

  # historical returns
  historyPrices['growth_7d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(7)



  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if largest_stocks_df.empty:
    largest_stocks_df = historyPrices
  else:
    largest_stocks_df = pd.concat([largest_stocks_df, historyPrices], ignore_index=False)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed


In [45]:
largest_stocks_df

,Open,High,Low,Close,Adj Close,Volume,Ticker,ticker_category,growth_7d
Date,,,,,,,,,
2013-12-26,37.200001,37.490002,37.169998,37.439999,31.468412,17612800,MSFT,LARGEST,NaN
2013-12-27,37.580002,37.619999,37.169998,37.290001,31.342333,14563000,MSFT,LARGEST,NaN
2013-12-30,37.220001,37.380001,36.900002,37.290001,31.342333,16290500,MSFT,LARGEST,NaN
2013-12-31,37.400002,37.580002,37.220001,37.410000,31.443182,17503500,MSFT,LARGEST,NaN
2014-01-02,37.349998,37.400002,37.099998,37.160000,31.233068,30632200,MSFT,LARGEST,NaN
...,...,...,...,...,...,...,...,...,...
2023-12-22,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707,LT.NS,LARGEST,1.022987
2023-12-26,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263,LT.NS,LARGEST,1.016588
2023-12-27,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266,LT.NS,LARGEST,1.016055


In [46]:
large_stocks_df = pd.DataFrame({'A' : []})
# Define the start and end dates
start_date = '2013-12-25'
end_date = '2024-01-01'
for i,ticker in enumerate(LARGE_STOCKS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,start=start_date, end=end_date,
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['ticker_category']= "LARGE"
  # historical returns
  historyPrices['growth_7d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(7)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if large_stocks_df.empty:
    large_stocks_df = historyPrices
  else:
    large_stocks_df = pd.concat([large_stocks_df, historyPrices], ignore_index=False)

0 PRX.AS


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 CDI.PA


2 AIR.PA


[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed


32 KOTAKBANK.NS


[*********************100%%**********************]  1 of 1 completed


33 TITAN.NS


[*********************100%%**********************]  1 of 1 completed


In [47]:
#Create Date column from index
large_stocks_df.reset_index(inplace=True)
largest_stocks_df.reset_index(inplace=True)

In [48]:
largest_stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80884 entries, 0 to 80883
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             80884 non-null  datetime64[ns]
 1   Open             80884 non-null  float64       
 2   High             80884 non-null  float64       
 3   Low              80884 non-null  float64       
 4   Close            80884 non-null  float64       
 5   Adj Close        80884 non-null  float64       
 6   Volume           80884 non-null  int64         
 7   Ticker           80884 non-null  object        
 8   ticker_category  80884 non-null  object        
 9   growth_7d        80653 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(2)
memory usage: 6.2+ MB


In [49]:
large_stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83968 entries, 0 to 83967
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             83968 non-null  datetime64[ns]
 1   Open             83968 non-null  float64       
 2   High             83968 non-null  float64       
 3   Low              83968 non-null  float64       
 4   Close            83968 non-null  float64       
 5   Adj Close        83968 non-null  float64       
 6   Volume           83968 non-null  int64         
 7   Ticker           83968 non-null  object        
 8   ticker_category  83968 non-null  object        
 9   growth_7d        83730 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(2)
memory usage: 6.4+ MB


In [50]:
a=large_stocks_df.groupby(by=['Date','ticker_category']).growth_7d.mean()
b=largest_stocks_df.groupby(by=['Date','ticker_category']).growth_7d.mean()
merged_df = pd.merge(a, b, on=['Date'])
merged_df

,growth_7d_x,growth_7d_y
Date,,
2013-12-26,NaN,NaN
2013-12-27,NaN,NaN
2013-12-30,NaN,NaN
2013-12-31,NaN,NaN
2014-01-01,NaN,NaN
...,...,...
2023-12-22,1.003014,1.013788
2023-12-26,1.013505,1.014900
2023-12-27,1.002167,1.014155


In [51]:
# Rename the columns for clarity
merged_df.rename(columns={'growth_7d_x': 'growth_7d_large', 'growth_7d_y': 'growth_7d_largest'}, inplace=True)

In [52]:
#  if growth_7d_large>growth_7d_largest put one in a new column called "Outperform" sum "Outperform" and divided by 2599

# Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP
outperform_count = merged_df[merged_df['growth_7d_large'] > merged_df['growth_7d_largest']].shape[0]

# Calculate the total number of trading days
total_trading_days = merged_df.shape[0]

# Calculate the percentage of days when the LARGE GROUP outperforms the LARGEST GROUP
outperform_percentage = round(outperform_count / total_trading_days * 100)

# Print the result
print(f"The LARGE GROUP outperforms the LARGEST GROUP on {outperform_percentage}% of days.")

The LARGE GROUP outperforms the LARGEST GROUP on 47% of days.


Question 4: Trying Another Technical Indicators strategy
What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?

First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31). If you encounter any difficulties running the Colab - you can download it using this link.

Let's assume you've learned about the awesome CCI indicator (Commodity Channel Index), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (Date.dt.dayofweek()==4).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS
1000 and received
10 to gross profit, if you received
20 to gross profit. You need to sum these results over all trades (460 times in 10 years).

Additional:

Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
are you still profitable on those trades?

In [55]:
m16 = pd.read_parquet(f"stocks_df_combined_trunc_2014_2023.parquet.brotli")

In [56]:
m17= m16[['Date','Adj Close_x','Ticker','cci','Weekday']].copy()
m17['gross profit']=0
m17.reset_index(inplace=True)
m17=m17.drop(labels='index',axis=1)
m17.loc[0,'gross_profit']=1000
m17

,Date,Adj Close_x,Ticker,cci,Weekday,gross profit,gross_profit
0,2014-01-02,31.233059,MSFT,57.700615,3,0,1000.0
1,2014-01-03,31.022930,MSFT,1.373763,4,0,NaN
2,2014-01-06,30.367352,MSFT,-96.631259,0,0,NaN
3,2014-01-07,30.602673,MSFT,-83.904297,1,0,NaN
4,2014-01-08,30.056356,MSFT,-147.855135,2,0,NaN
...,...,...,...,...,...,...,...
80757,2023-12-22,3477.949951,LT.NS,70.767162,4,0,NaN
80758,2023-12-26,3490.050049,LT.NS,99.598220,1,0,NaN
80759,2023-12-27,3544.000000,LT.NS,130.401152,2,0,NaN
80760,2023-12-28,3518.050049,LT.NS,106.774509,3,0,NaN


In [57]:
m17['buy']=0
m17.loc[(m17['cci']>200) & (m17['Weekday']==4),'buy']=1
m17['buy'].sum()

460

In [58]:
lst_buy=list(m17.loc[m17['buy']==1].index)

on_trade= 0
idx_trade = 0
price_in=0
profit = 1000
ticker= m17.iloc[0]['Ticker']
operation_counter = 0
t1=time.time()
m17['gross_profit2']=np.NaN

for idx in lst_buy:
  idx_trade=idx
  on_trade = 1
  price_in =m17.iloc[idx]['Adj Close_x']
  ticker=m17.iloc[idx]['Ticker']
  # print(idx,profit)
  for jdx in range(idx,m17.shape[0]):
    if on_trade ==1 and idx_trade+5==jdx  and m17.iloc[jdx]['Ticker'] == ticker:
      price_out= m17.iloc[jdx]['Adj Close_x']
      on_trade = 0
      profit = price_out/price_in*profit
      operation_counter+=1
      m17.loc[m17.iloc[jdx].name,'gross_profit2']=profit
      break
    elif (not m17.iloc[jdx]['Ticker'] == ticker) and on_trade ==1:
      price_out= m17.iloc[jdx-1]['Adj Close_x']
      on_trade = 0
      profit = price_out/price_in*profit
      # print(f" pout{price_out} pin {price_in} {profit}")
      operation_counter+=1
      m17.loc[m17.iloc[jdx].name,'gross_profit2']=profit
      break

In [59]:
print(f"The number of operations is {operation_counter}, the time enlapsed is {time.time()-t1:.2f}, and the profit final {profit/1000:.0f}$ thousand ")

The number of operations is 460, the time enlapsed is 11.79, and the profit final 2$ thousand 


**Question 5**
Sector Focus:

1: Analyze historical IPO performance by sector (technology, healthcare, industrials, etc.). Look for sectors with consistently higher returns. Public news, sector-specific publications, and market analysis websites provide this data.
Why: Different sectors have varying risk profiles and growth potential. Focusing on sectors with favorable trends increases your chances of success.
Company Fundamentals:

2: Scrutinize the company's prospectus (available on sites like the SEC's EDGAR database):
Growth potential: Revenue growth, profit margins, market size, addressable market.
Financial Health: Debt levels, cash flow, and profitability.
Management: Experience of management team, track record of previous ventures.
Why: Strong fundamentals indicate the company's ability to perform well in the long term, even after the initial IPO hype fades.
Underwriter Reputation:

3: Look at the historical performance of IPOs underwritten by specific investment banks. This information is available on financial websites and IPO databases
Why: Reputable underwriters have a track record of bringing successful companies to market and accurately pricing IPOs.
Valuation and Market Sentiment:

4: Consider the IPO's valuation compared to similar, publicly traded companies. Look for overvaluation red flags. Assess overall market sentiment through financial news and economic indicators.
Why: Investing in overvalued IPOs increases your risk, while favorable market sentiment can boost returns.
Post-IPO Lockup Period:

5: Be mindful of the lockup period (when company insiders are restricted from selling shares), usually around 180 days.
Why: A significant drop in share price after the lockup period expires could signal insider pessimism about the company's future.
Important Considerations:

Diversify: Don't pour everything into a single IPO. Spread your investments across promising candidates.
Don't chase hype: IPOs can be heavily marketed. Focus on the fundamental data, not just the buzz.
Long-term perspective: IPOs can be volatile in the short term. Consider a longer investment horizon.